# 导入必要的包

In [38]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l
import pandas as pd # 读取数据
from torch import nn

# 读取并加载数据


In [40]:
def load_array(data_arrays, batch_size, is_train=True):
    """
    构造一个 PyTorch 数据迭代器。

    根据给定的数据数组创建一个 `DataLoader`，用于批量加载数据，支持随机打乱数据顺序。

    参数:
        data_arrays (tuple[torch.Tensor, ...]): 包含特征和标签的元组，每个元素是一个 `torch.Tensor`。
        batch_size (int): 每个小批量的样本数量。
        is_train (bool): 是否将数据打乱（`True` 表示训练模式，数据会随机打乱；`False` 表示测试模式，数据按顺序加载）。

    返回:
        torch.utils.data.DataLoader: 一个数据加载器对象，用于按批量加载数据。
    """
    # 使用 TensorDataset 将特征和标签封装成一个数据集对象
    dataset = data.TensorDataset(*data_arrays)

    # 创建 DataLoader，指定批量大小和是否随机打乱数据
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

# 检验迭代器是否正常工作

In [41]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[-0.4364,  0.2379, -0.8663],
         [-0.4827, -0.5954,  0.3239],
         [-0.8548,  0.3612,  0.9981],
         [-0.2216,  0.8152,  1.5884],
         [ 0.5606, -2.8520,  0.1266],
         [-0.6191, -0.4520, -0.0456],
         [-0.2312, -1.1332,  0.2500],
         [ 1.0375,  1.3158,  0.8466],
         [ 0.9669, -0.8326,  1.0349],
         [-0.4858, -1.0137,  1.8953]]),
 tensor([[-5.7090, -0.7990],
         [ 0.8442,  2.0223],
         [ 3.8831, -1.2231],
         [ 8.4354, -2.7539],
         [ 1.8268,  9.6846],
         [-1.5062,  1.5518],
         [ 0.9539,  3.8662],
         [ 6.8066, -4.4758],
         [ 7.7479,  2.8347],
         [ 9.6270,  3.4516]])]

# 定义线性模型

In [42]:
in_features = 3
out_features = 2
net = nn.Sequential(nn.Linear(in_features, out_features))
# nn是神经网络的缩写
# Sequential是一个有序容器，网络层将按照在传入Sequential的顺序依次被添加到计算图中执行
# Linear是一个全连接层，它接收一个输入张量，然后计算输出

# 初始化模型参数

In [43]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0., 0.])

# 定义损失函数

In [44]:
loss = nn.MSELoss()

# 定义优化算法

In [45]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
# parameters()返回一个包含模型所有参数的迭代器

# 训练

In [51]:
num_epochs = 3  # 设置训练的总轮数
for epoch in range(num_epochs):  # 遍历每一轮训练
    for X, y in data_iter:  # 遍历每个小批量数据
        # 计算当前批量数据的损失
        l = loss(net(X), y)

        # 梯度清零，防止累积
        trainer.zero_grad()  

        # 计算梯度
        l.backward()  

        # 根据梯度更新参数
        trainer.step()  

    # 在每轮训练结束后，计算整个数据集上的损失
    l = loss(net(features), labels)  

    # 打印当前轮次和对应的损失
    print(f'epoch {epoch + 1}, loss {l:f}')


epoch 1, loss 0.000100
epoch 2, loss 0.000100
epoch 3, loss 0.000100
